<a href="https://colab.research.google.com/github/Kerimova13/Helloworld/blob/master/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Устанавливаем библиотеку lightfm
pip install lightfm nmslib

In [0]:
#Скачиваем данные
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2020-03-21 11:48:53--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip.3’

ml-latest-small.zip 100%[===================>] 955.28K   974KB/s    in 1.0s    

2020-03-21 11:48:54 (974 KB/s) - ‘ml-latest-small.zip.3’ saved [978202/978202]



In [0]:
#Разархивируем данные
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
replace ml-latest-small/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


#Импортируем модули из библиотеки

In [0]:
from lightfm import LightFM
import nmslib
import pandas as pd
from scipy.sparse import coo_matrix

# Вспомогательные функции

In [0]:
#Это вспомогательная функция, которая преобразует данные к удобному виду
def update_data(movies,ratings):
  #функция для преобразования данных
  ratings['ID'] = ratings.movieId.astype('category').cat.rename_categories(range(1, ratings.movieId.nunique()+1)).astype(int)
  movies = pd.merge(movies,ratings,how='inner',on='movieId').drop_duplicates('movieId')
  ratings.rating = ratings.rating.apply(lambda x: 1 if x> 4 else 0)
  ratings_sparse = coo_matrix((ratings.rating, (ratings.userId,ratings.ID)))
  return movies[['title','genres','ID']].set_index('ID'),ratings[['userId','ID','rating']],ratings_sparse

In [0]:
# Функция для работы с поиском
def init_nms (item_embeddings):
  # initialize a new nmslib index, using a HNSW index on Cosine Similarity
  nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
  nms_idx.addDataPointBatch(item_embeddings)
  nms_idx.createIndex(print_progress=True)
  return nms_idx

# Загружаем данные


In [0]:
movies =  pd.read_csv('ml-latest-small/movies.csv')
ratings =  pd.read_csv('ml-latest-small/ratings.csv')
#Преобразуем данные
movies,ratings,ratings_sparse = update_data(movies,ratings)

#Чтобы вывести данные, нужно ввести имя таблицы и запустить ячейку кликнув на значок play 



In [0]:
# Вывести таблицу movies путем ввода её названия movies
# Ваш код тут:
movies

,title,genres
ID,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
9720,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9721,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9722,Flint (2017),Drama


## **Напишем код, чтобы вывести первые 5 фильмов**

In [0]:
# Чтобы вывести первые элементы, нужно воспользоваться функцией head 
# Ваш код тут:
movies.head(6)

,title,genres
ID,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),Action|Crime|Thriller


## Напишите код, чтобы вывести оценки

In [0]:
# Оценки хранятся в таблице ratings, чтобы вывести первые элементы, нужно воспользоваться функцией head
# Ваш код тут:
ratings.head(6)

,userId,ID,rating
0,1,1,0
1,1,3,0
2,1,6,0
3,1,44,1
4,1,47,1
5,1,63,0


# Смотрим фильмы

In [0]:
#Тут ничего писать не надо, 
#давайте посмотрим код!
#@title Выберем фильм
movie_name = "Matrix" #@param {type:"string"}
movies[movies['title'].str.find(movie_name)>=0]

,title,genres
ID,,
1939,"Matrix, The (1999)",Action|Sci-Fi|Thriller
4346,"Matrix Reloaded, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX
4632,"Matrix Revolutions, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX


# Время магии! Давайте создадим модель




In [0]:
#Создадим модель LightFM с аргументами (loss=LOSS, no_components=NO_COMPONENTS)
LOSS ="warp" # Алгоритм обучения 
NO_COMPONENTS = 30 # Размер пространства, Введите ваш код тут
model = LightFM(loss=LOSS, no_components=NO_COMPONENTS) # Создаём модель тут, введите ваш код тут model(...)

# Обучим модель

In [0]:
# Обучаем модель, нужно вызвать функцию .fit у нашей модели model с аргументами (data,epochs)
# Наши данные - это рейтинги пользователей ratings_sparse
data = ratings_sparse 
# обучаем модель, введите ваш код тут model.fit(...)
model.fit(data)

# Используем модель

In [0]:
#Тут мы получаем новое представление наших данных
_, item_embeddings = model.get_item_representations()
#Построим поиск
nms_idx = init_nms(item_embeddings)

# Вспомогательная функция


In [0]:
#Вспомогательная функция для поиска ближайших фильмов
def nearest_movies_nms(movie_id, index, n=10):
    #Возвращает ближайшие фильмы
    nn = index.knnQuery(item_embeddings[movie_id], k=n)
    return nn

# Выбираем фильм для рекомендации

In [0]:
#@title Выберем фильм
movie_name = "Inception" #@param {type:"string"}
ID = 7356 #@param {type:"number"}
movies[movies['title'].str.find(movie_name)>=0]

,title,genres
ID,,
7356,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX


# Получим рекомендации

In [0]:
# Получаем рекомендации
rec = nearest_movies_nms(ID,nms_idx)[0]

# Выведем наши рекомендации

In [0]:
# воспользуемся таблицей movies и функцией .loc с аргументами [rec], тут нужно использовать
# квадратные скобки
# Ваш код тут
movies.loc[rec]

,title,genres
ID,,
3570,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy
6506,Harry Potter and the Order of the Phoenix (2007),Adventure|Drama|Fantasy|IMAX
5306,"Bill Cosby, Himself (1983)",Comedy|Documentary
2016,Tarzan (1999),Adventure|Animation|Children|Drama
515,Pretty Woman (1990),Comedy|Romance
2671,Gladiator (2000),Action|Adventure|Drama
7676,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX
3828,Star Wars: Episode II - Attack of the Clones (...,Action|Adventure|Sci-Fi|IMAX
6469,Jonestown: The Life and Death of Peoples Templ...,Documentary


# Получили плохие результаты, давайте их улучшим!

## Как можно улучшить нашу модель?


In [0]:
#@title Выберем кол-во эпох для обучения
# Обучаем модель, нужно вызвать функцию .fit у нашей модели model с аргументами (data,epochs)
epochs =  100#@param {type:"number"}
model.fit(data,epochs=epochs) # обучаем модель

In [0]:
import random
random.seed(42)

In [0]:
#Тут мы получаем новое представление наших данных
_, item_embeddings = model.get_item_representations()
#Построим поиск
nms_idx = init_nms(item_embeddings)

In [0]:
# Получаем рекомендации
rec = nearest_movies_nms(ID,nms_idx)[0]

In [0]:
movies.loc[rec]

,title,genres
ID,,
7356,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX
8359,Interstellar (2014),Sci-Fi|IMAX
6694,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX
6994,Inglourious Basterds (2009),Action|Drama|War
8046,Django Unchained (2012),Action|Drama|Western
7242,Shutter Island (2010),Drama|Mystery|Thriller
6906,Gran Torino (2008),Crime|Drama
8355,"Grand Budapest Hotel, The (2014)",Comedy|Drama
8235,Gravity (2013),Action|Sci-Fi|IMAX


In [0]:
movies.loc[rec]

,title,genres
ID,,
1939,"Matrix, The (1999)",Action|Sci-Fi|Thriller
3137,Memento (2000),Mystery|Thriller
911,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi
2980,"Crouching Tiger, Hidden Dragon (Wo hu cang lon...",Action|Drama|Romance
3634,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
1503,Saving Private Ryan (1998),Action|Drama|War
2225,Fight Club (1999),Action|Crime|Drama|Thriller
794,Die Hard (1988),Action|Crime|Thriller
4132,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy


# А что если изменить размерность нашего пространства с 30 до 1


In [0]:
NO_COMPONENTS =   100#@param {type:"number"}
model = LightFM(loss=LOSS, no_components = NO_COMPONENTS) # Создаём модель тут, введите ваш код тут
epochs =  100#@param {type:"number"}
model.fit(data,epochs=epochs) # обучаем модель, введите ваш код тут

In [0]:
#Тут мы получаем новое представление наших данных
_, item_embeddings = model.get_item_representations()
#Построим поиск
nms_idx = init_nms(item_embeddings)

In [0]:
# Получаем рекомендации
rec = nearest_movies_nms(ID,nms_idx)[0]

In [0]:
nearest_movies_nms(ID,nms_idx)[1]

array([0.        , 0.19141555, 0.22392285, 0.28938806, 0.28994387,
       0.29812783, 0.3051318 , 0.31696337, 0.32168782, 0.33135068],
      dtype=float32)

In [0]:
movies.loc[rec]

,title,genres
ID,,
7356,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX
6694,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX
7242,Shutter Island (2010),Drama|Mystery|Thriller
9417,Split (2017),Drama|Horror|Thriller
8355,"Grand Budapest Hotel, The (2014)",Comedy|Drama
7023,Up (2009),Adventure|Animation|Children|Drama
6994,Inglourious Basterds (2009),Action|Drama|War
8740,The Hateful Eight (2015),Western
7751,"Dark Knight Rises, The (2012)",Action|Adventure|Crime|IMAX


Что можно сделать?


1.   Изменить параметры размерности!
2.   Изменить критерий оценки

